# Assignment

In this assignment, we develop on the ideas of linear regression we learned in the lecture to train multiple linear regression models and compare them. Recall that the HSB2 data contains students' scores in five different subjects.

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['figure.figsize'] = [10, 7]
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as statsmodels
import statsmodels.formula.api as sm
#import the model from sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
hsb2 = pd.read_csv('./data/hsb2.csv')
X = hsb2[['read', 'write', 'math', 'science']]
Y = hsb2['socst']
X.head()

,read,write,math,science
0,57,52,41,47
1,68,59,53,63
2,44,33,54,58
3,63,44,47,53
4,47,52,57,53


In [4]:
#type(Y)
Y.head()

0    57
1    61
2    31
3    56
4    61
Name: socst, dtype: int64

In [5]:
hsb2.head()

,id,female,race,ses,schtyp,prog,read,write,math,science,socst
0,70,0,4,1,1,1,57,52,41,47,57
1,121,1,4,2,1,3,68,59,53,63,61
2,86,0,4,3,1,1,44,33,54,58,31
3,141,0,4,3,1,3,63,44,47,53,56
4,172,0,4,2,1,2,47,52,57,53,61


1 / 9
- Use the reading, writing, math and science scores to predict a student's score in social studies. Train your model using `LinearRegression` in `sklearn`. Add the model's predictions to the data as a new column called `socst_pred_skl`. <span style="color:red" float:right>[5 point] [of 28 pts] </span>

In [7]:
#initialize the model 
model = LinearRegression()
#fit the data 
    #input is reading, writing, math, and science scores (X)
    #output is scores in social studies (Y)
model.fit(X, Y)
#Predictions are added as a new column
hsb2["socst_pred_skl"] = model.predict(X)
#Print the linear model
print("The model is at y=",model.coef_,"x_f + ", model.intercept_)
#Print the score (aka mean accuracy) of the model
print("The accuracy is ", round(model.score(X,Y)*100,2))

The model is at y= [ 0.38075199  0.37518056  0.13222368 -0.02794164] x_f +  7.206027496188192
The accuracy is  47.72


2 / 9
- Train the same model a second time, but this time use `sm.ols` in `statsmodels` to do it. Add the model's predictions to the data as a new column called `socst_pred_ols`. <span style="color:red" float:right>[3 point] of 28 pts </span>

In [9]:
# Add a constant term to X (required for statsmodels OLS)
    #renamed X because it added a constant, which threw off the concatenation below
X_OLS = statsmodels.add_constant(X)

#initialize the model 
model_statsmodels = statsmodels.regression.linear_model.OLS(Y,X_OLS)

#fit the data 
    #input is reading, writing, math, and science scores (X)
    #output is scores in social studies (Y)
results = model_statsmodels.fit()

#Predictions are added as a new column
hsb2["socst_pred_ols"] = results.predict(X_OLS)

#Print the linear model
print("The model is at y=",results.params[1:],"x_f + ", results.params[0])

#Print the score (aka mean accuracy) of the model
print("The accuracy is ", round(results.rsquared*100,2))

The model is at y= read       0.380752
write      0.375181
math       0.132224
science   -0.027942
dtype: float64 x_f +  7.20602749618822
The accuracy is  47.72


C:\Users\Alissa\AppData\Local\Temp\ipykernel_7988\2004242341.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("The model is at y=",results.params[1:],"x_f + ", results.params[0])


3 / 9
- Check that the predictions from the two models are the same. <span style="color:red" float:right>[1 point] of 28 pts</span>

In [11]:
#Print description statistics to compare the predictions
hsb2[["socst_pred_skl","socst_pred_ols"]].describe()


,socst_pred_skl,socst_pred_ols
count,200.000000,200.000000
mean,52.405000,52.405000
std,7.415957,7.415957
min,36.947826,36.947826
25%,46.898075,46.898075
50%,52.549074,52.549074
75%,58.140526,58.140526
max,67.765579,67.765579


4 / 9
- Print the model summary from the model we trained with `sm.ols`. The formula uses a format similar to `Y ~ X_1 + X_2`. Interpret the model's coefficients and write a few brief sentences about what the model is telling us. <span style="color:red" float:right>[5 point] of 28 pts</span>

In [13]:
#Print the model summary from the model
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  socst   R-squared:                       0.477
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     44.49
Date:                Tue, 18 Mar 2025   Prob (F-statistic):           1.65e-26
Time:                        14:41:41   Log-Likelihood:                -693.15
No. Observations:                 200   AIC:                             1396.
Df Residuals:                     195   BIC:                             1413.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.2060      3.611      1.995      0.0

Because the Prob of the F-statistic is so low (< 0.05), it suggests this model is highly significant. And because two coefficients of read and write have p-value < 0.05, these two features are also significant. But, because two features have p-values > 0.05 (aka math and science), these two features are not significant, so the model may be improved by removing these two features. There is some mild concern that the residuals are not normally distributed based on a negative skew and a kurtosis less than 3, especially since this P of both Omnibus and JB are less than 0.05, which suggests we can reject the null that they are normally distributed. And with a high Condition number, there is a likelihood that some of these features are correlated with each other. 

5 / 9 
- Find the model's coefficents using the closed-form solution for linear regression:  <span style="color:red" float:right>[3 point] of 28 pts</span>

$${b} = ({X'}{X})^{-1}{X'}{Y}$$ 

To make it easy, we have created a matrix `M` which is the matrix $X$ with an extra column of 1s for the intercept term. Make sure the results match what's reported in the above model summary. HINT: You can use `np.linalg.inv` to invert a matrix, `np.matmul` to multiply two matrices, and the `np.transpose()` method to take the transpose of a matrix. 

In [16]:
#concatenate: join a series of arrays along axis = 1
    #1st: array filled with ones with a length the same as Y --> reshaped to be (-1):Auto-calculate rows and +1 columns
    #2nd: array of values inside X
M = np.concatenate([np.ones(len(Y)).reshape(-1, 1), X.values], axis = 1)
print(M[:4, :]) #print first 4 rows and all (6)columns
#M.shape #200 rows and 6 columns
    #I don't think there should be 6 col, should be 5: 4 features + 1 constant

#to find the model's coefficients using the formula (vs. the two models)
transposed_M = np.transpose(M)
#print(transposed_M)
multiplied = np.matmul(transposed_M, M)
#print(multiplied)
parentheses = np.linalg.inv(multiplied)
#print(parentheses)
beta = np.matmul(np.matmul(parentheses, transposed_M), Y)
beta

[[ 1. 57. 52. 41. 47.]
 [ 1. 68. 59. 53. 63.]
 [ 1. 44. 33. 54. 58.]
 [ 1. 63. 44. 47. 53.]]


array([ 7.2060275 ,  0.38075199,  0.37518056,  0.13222368, -0.02794164])

6 / 9 
- The `fit` method used by `sm.ols` has an argument called `normalize`. Use it to train the same model but using the normalized features instead. What changes do you notice in the model summary? <span style="color:red" float:right>[2 point]</span>

In [18]:
#Apply normalization to data
X_copy = X.copy()
X_normalized = X.copy()
for each in X_copy.columns: 
    X_normalized[each]= ( X_copy[each] - np.mean(X_copy[each]) )/ np.std(X_copy[each])

# Add a constant term to X (required for statsmodels OLS)
    #renamed X because it added a constant, which threw off the concatenation below
X_OLS_normalized = statsmodels.add_constant(X_normalized)

#initialize the model 
model_statsmodels_normalized = statsmodels.regression.linear_model.OLS(Y,X_OLS_normalized)

#fit the data 
    #input is reading, writing, math, and science scores (X)
    #output is scores in social studies (Y)
results_normalized = model_statsmodels_normalized.fit()

#Predictions are added as a new column
hsb2["socst_pred_ols"] = results_normalized.predict(X_OLS_normalized)

#Print the linear model
print("The model is at y=",results_normalized.params[1:],"x_f + ", results_normalized.params[0])

#Print the score (aka mean accuracy) of the model
print("The accuracy is ", round(results_normalized.rsquared*100,2))

#Print the summary
print(results_normalized.summary())

The model is at y= read       3.894054
write      3.547280
math       1.235630
science   -0.275955
dtype: float64 x_f +  52.405
The accuracy is  47.72
                            OLS Regression Results                            
Dep. Variable:                  socst   R-squared:                       0.477
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     44.49
Date:                Tue, 18 Mar 2025   Prob (F-statistic):           1.65e-26
Time:                        14:41:41   Log-Likelihood:                -693.15
No. Observations:                 200   AIC:                             1396.
Df Residuals:                     195   BIC:                             1413.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|     

C:\Users\Alissa\AppData\Local\Temp\ipykernel_7988\3611962062.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("The model is at y=",results_normalized.params[1:],"x_f + ", results_normalized.params[0])


I don't notice too many relatively large differences between the two models of not normalized vs normalized features. They share similar positive and negative qualities. The only relatively big difference is that the condition number has minimized to 3, which suggests features no longer share any correlation. 

7 / 9 

NOTE: We say that ordinary least squares is **invariant to normalization**, while other methods we learn later such as lasso or ridge regression are not.

- Train a new model with `sm.ols` but this time only include the reading and writing scores as features. Decide if you should use a model *with or without* interactions. HINT: Change the formula from `Y ~ X_1 + X_2` to `Y ~ X_1 * X_2` and the new model will estimate the main effects $\beta_\text{read}$ and $\beta_\text{write}$ and their interaction $\beta_\text{read:write}$. <span style="color:red" float:right>[2 point]</span>

In [21]:
#initialize the model 
    #Col names: read 	write 	math 	science 	socst
    #input is reading, writing, math, and science scores (X)
    #output is scores in social studies (Y)
model_formula = sm.ols(formula = 'socst ~ read * write', data=hsb2)

#fit the data 
results_formula = model_formula.fit()

#Print the summary
print(results_formula.summary())

                            OLS Regression Results                            
Dep. Variable:                  socst   R-squared:                       0.472
Model:                            OLS   Adj. R-squared:                  0.464
Method:                 Least Squares   F-statistic:                     58.32
Date:                Tue, 18 Mar 2025   Prob (F-statistic):           5.45e-27
Time:                        14:41:41   Log-Likelihood:                -694.21
No. Observations:                 200   AIC:                             1396.
Df Residuals:                     196   BIC:                             1410.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     15.2578     17.391      0.877      0.3

Because the p of the read:write interaction is > 0.05, it suggests that we should use a model without this interaction. This and the p has increased for each of the coefficients, which shows including this interaction would not be worthwhile. 

8 / 9 

- So far we've only had models with numeric features, but it's possible to also include **categorical features**. Add the term `C(prog)` to the formula. This will add the feature `prog` to the model as a categorical feature. This feature states which of 3 after-school program a student participated in. <span style="color:red" float:right>[2 point]</span>

In [24]:
#initialize the model 
    #Col names: read 	write 	math 	science 	socst
        #add the categorical feature: prog
    #input is reading, writing, math, and science scores (X)
    #output is scores in social studies (Y)
#model_formula = sm.ols(formula = 'socst ~ read + write + math + science + C(prog)', data=hsb2)
model_formula = sm.ols(formula = 'socst ~ read + write + C(prog)', data=hsb2)


#fit the data 
results_formula = model_formula.fit()

#Print the summary
print(results_formula.summary())

                            OLS Regression Results                            
Dep. Variable:                  socst   R-squared:                       0.495
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     47.78
Date:                Tue, 18 Mar 2025   Prob (F-statistic):           5.84e-28
Time:                        14:41:41   Log-Likelihood:                -689.69
No. Observations:                 200   AIC:                             1389.
Df Residuals:                     195   BIC:                             1406.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       13.4029      3.755      3.569   

9 / 9 

- Report the important results in the model summary. <span style="color:red" float:right>[5 point]</span>

- <b>R-squared</b>: Is 0.495 --> Okay since it's in the middle of [0, 1]
- <b>Adj. R-squared</b>: Is 0.485 --> Okay since it's in the middle of [0, 1]
- <b>F-statistic</b>: Is 47.78 
- <b>Prob (f-statistic)</b>: 5.84e-28 --> Really good (significant) since it's < alpha of 0.05. 
- <b>Log-Likelihood</b>: -689.69 --> Relatively similar to other models we've run
- <b>AIC</b>: 1389 --> Relatively similar to other models we've run
- <b>BIC</b>: 1406 --> Relatively similar to other models we've run
- <b>Coefficients</b>:
    - Intercept: 0.000 --> Really good (significant) since it's < alpha of 0.05
    - C: 0.184 and 0.109 --> Not great (not significant) since it's > alpha of 0.05
    - Read: 0.000 --> Really good (significant) since it's < alpha of 0.05
    - Write: 0.000 --> Really good (significant) since it's < alpha of 0.05
 
- <b>Omnibus</b>: 6.641 --> because this is greater than zero, suggests residuals are not perfectly normally distributed
- <b>Prob(Omnibus)</b>: Is 0.036 --> Alright (somewhat significant) since it's < alpha of 0.05, so we can reject the null and say that the data is not normally distributed, but it's close
- <b>Skew</b>: -0.449 --> Since it's negative, suggests that residuals are mildly skewed to the left
- <b>Kurtosis</b>: 2.881 --> Since it's < 3, suggests that there are tails mildly smaller than a normal distribution
- <b>Durbin-Watson</b>: 1.943 --> Since it's <2, suggests that is some mild positive autocorrelation
- <b>Jarque-Bera</b>: 6.837 --> because this is greater than zero, suggests residuals are not perfectly normally distributed
- <b>Prob(JB)</b>: 0.0328 --> Alright (somewhat significant) since it's < alpha of 0.05, so we can reject the null and say that the data is not normally distributed, but it's close
- <b>Cond. No</b>: 527 --> since this is higher than 30, suggests multicollinearity might be a problem (aka some of the features are correlated with each other)

# End of assignment